In [2]:
%reset -f

import sys
import os
import bz2
import csv
import pandas as pd
import ast
# import json
import ujson
import numpy
import tqdm

def get_counts(directory, cam):
    # Dictionary zur Speicherung der DataFrames
    dataframes = {}

    # Durchsuche das Verzeichnis nach CSV-Dateien mit "counts" im Namen
    for filename in os.listdir(directory):
        if filename.endswith(".csv") and "counts" in filename:
            # Importiere die CSV-Datei und speichere sie im Dictionary
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['filename'] = filename
            df['SigmaIOU'] = df['filename'].apply(lambda x: x[-19:-17])
            dataframes[filename] = df
    counts = pd.concat(dataframes.values(), ignore_index=True)
    counts_sum = counts.groupby(['SigmaIOU', 'flow'])['count'].sum().reset_index()
    count_pivot = counts_sum.pivot(index='SigmaIOU', columns='flow', values='count')
    count_pivot['Lauf'] = cam
    
    count_pivot.to_csv(directory + cam + '_' + 'Counts_sum.csv', index=False)

    return count_pivot

counts_Leipzig_OTC07 = get_counts(directory = "Z:/04_Daten/GroundThruth/Leipzig/OTC07/vary_SigmaL/", cam = 'Leipzig_OTC07')
counts_Leipzig_OTC19 = get_counts(directory = "Z:/04_Daten/GroundThruth/Leipzig/OTC19/vary_SigmaL/", cam = 'Leipzig_OTC19')
counts_KA_OTC13 = get_counts(directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC13_23-10-22_17-00-00/vary_SigmaL/", cam = 'KA_OTC13')
counts_KA_OTC23 = get_counts(directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC07_23-09-19_17-00-00/vary_SigmaL/", cam = 'KA_OTC23')
# counts_COP_OTC23 = get_counts(directory = "Z:/04_Daten/GroundThruth/Kopenhagen/otcamera23_FR20_2023-10-06_16-30-00/vary_SigmaIOU/", cam = 'COP_OTC23')




In [64]:
directory = 'Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC13_23-10-22_17-00-00'
Fall = 'OTCamera07_FR20_2023-09-19_17-00-00_Sued'

def events_to_df(filepath, key = 'event_list'):
    # Open otevents-file
    with bz2.open(filepath, "rt", encoding="UTF-8") as file:
        dictfile = ujson.load(file)
    EVENTS = pd.DataFrame.from_dict(dictfile[key])
    return EVENTS

GT_events = pd.DataFrame()

for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.otgtevents'):
                path = os.path.join(root, file)
                with bz2.open(path, "rt", encoding="UTF-8") as file:
                    dictfile = ujson.load(file)
                EVENTS = pd.DataFrame.from_dict(dictfile['event_list'])
                # EVENTS['File'] = file
                
                SECTIONS = pd.DataFrame.from_dict(dictfile['sections']).rename(columns = {'id': 'section_id', 'name': 'section_name'})
    
GT = pd.merge(EVENTS, SECTIONS, on = 'section_id')    


GT = GT[['road_user_id', 'road_user_type', 'section_name']]

GT = GT.pivot(index='road_user_id', columns = 'section_name', values='road_user_type')

# GT_events = GT_events.drop_duplicates(subset='road_user_id', keep='first')